In [164]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [165]:
# Read in crsp daily data
df = pd.read_csv("subset.csv", index_col=0)

# Drop columns
df = df.drop(columns=['shrcd', 'exchcd', 'shrout'])

# Clean date variables
df['caldt'] = pd.to_datetime(df['caldt'])
df['year'] = df['caldt'].dt.year
df['month'] = df['caldt'].dt.month

# Read in benchmark data
bmk = pd.read_csv("bmk.csv")

# Clean date variables
bmk['caldt'] = pd.to_datetime(bmk['caldt'])

# Merge
df = df.merge(bmk, on='caldt', how='left')

df = df.drop_duplicates().reset_index(drop=True)

df

,permno,caldt,ticker,prc,ret,year,month,mkt
0,10001,2010-01-04,EGAS,10.25000,-0.004854,2010,1,NaN
1,10001,2010-01-05,EGAS,10.19000,-0.005854,2010,1,0.002561
2,10001,2010-01-06,EGAS,10.31000,0.011776,2010,1,0.001954
3,10001,2010-01-07,EGAS,9.96000,-0.033948,2010,1,0.003449
4,10001,2010-01-08,EGAS,10.34000,0.038153,2010,1,0.003587
...,...,...,...,...,...,...,...,...
13458532,93436,2023-12-22,TSLA,252.53999,-0.007701,2023,12,0.002534
13458533,93436,2023-12-26,TSLA,256.60999,0.016116,2023,12,0.004689
13458534,93436,2023-12-27,TSLA,261.44000,0.018822,2023,12,0.002297
13458535,93436,2023-12-28,TSLA,253.17999,-0.031594,2023,12,-0.000218


In [166]:
# Compute residuals to mkt factor (IWV daily returns)
window = 20

grouped = df.groupby('permno')

roll_mean_ret = grouped['ret'].rolling(window).mean().reset_index(drop=True)
roll_mean_mkt = grouped['mkt'].rolling(window).mean().reset_index(drop=True)

roll_cov = grouped[['mkt', 'ret']].apply(
    lambda x: x['ret'].rolling(window).cov(x['mkt'])
).reset_index(drop=True)

roll_var = grouped['mkt'].rolling(window).var().reset_index(drop=True)

beta = roll_cov / roll_var

alpha = roll_mean_ret - beta * roll_mean_mkt

fitted = alpha + beta * df['mkt']

residuals = df['ret'] - fitted

df['res'] = residuals

df

,permno,caldt,ticker,prc,ret,year,month,mkt,res
0,10001,2010-01-04,EGAS,10.25000,-0.004854,2010,1,NaN,NaN
1,10001,2010-01-05,EGAS,10.19000,-0.005854,2010,1,0.002561,NaN
2,10001,2010-01-06,EGAS,10.31000,0.011776,2010,1,0.001954,NaN
3,10001,2010-01-07,EGAS,9.96000,-0.033948,2010,1,0.003449,NaN
4,10001,2010-01-08,EGAS,10.34000,0.038153,2010,1,0.003587,NaN
...,...,...,...,...,...,...,...,...,...
13458532,93436,2023-12-22,TSLA,252.53999,-0.007701,2023,12,0.002534,-0.011243
13458533,93436,2023-12-26,TSLA,256.60999,0.016116,2023,12,0.004689,0.009250
13458534,93436,2023-12-27,TSLA,261.44000,0.018822,2023,12,0.002297,0.016981
13458535,93436,2023-12-28,TSLA,253.17999,-0.031594,2023,12,-0.000218,-0.028194


In [167]:
# Calculate momentum signal

# Log Returns
df['logret'] = np.log1p(df['ret'])

# Momentum from t-6 to t-1
df['mom'] = df.groupby('permno')['logret'].rolling(window*11,window*11).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(window)

df

,permno,caldt,ticker,prc,ret,year,month,mkt,res,logret,mom
0,10001,2010-01-04,EGAS,10.25000,-0.004854,2010,1,NaN,NaN,-0.004866,NaN
1,10001,2010-01-05,EGAS,10.19000,-0.005854,2010,1,0.002561,NaN,-0.005871,NaN
2,10001,2010-01-06,EGAS,10.31000,0.011776,2010,1,0.001954,NaN,0.011707,NaN
3,10001,2010-01-07,EGAS,9.96000,-0.033948,2010,1,0.003449,NaN,-0.034538,NaN
4,10001,2010-01-08,EGAS,10.34000,0.038153,2010,1,0.003587,NaN,0.037443,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13458532,93436,2023-12-22,TSLA,252.53999,-0.007701,2023,12,0.002534,-0.011243,-0.007731,0.683641
13458533,93436,2023-12-26,TSLA,256.60999,0.016116,2023,12,0.004689,0.009250,0.015988,0.650205
13458534,93436,2023-12-27,TSLA,261.44000,0.018822,2023,12,0.002297,0.016981,0.018647,0.691532
13458535,93436,2023-12-28,TSLA,253.17999,-0.031594,2023,12,-0.000218,-0.028194,-0.032104,0.690453


In [168]:
# Compute Alphas

df['ic'] = (
    df.groupby('permno')[['mom', 'res']]
    .apply(lambda group: group['mom'].rolling(22, 22).corr(group['res']))
    .reset_index(level=0, drop=True)
)

df['res_vol'] = df.groupby('permno')['res'].rolling(22,22).std().reset_index(drop=True)

df = df.dropna().reset_index(drop=True)
df['z_mom'] = df.groupby('caldt')['mom'].apply(zscore).reset_index(drop=True)

df['alpha'] = df['ic'] * df['res_vol'] * df['z_mom']

df

,permno,caldt,ticker,prc,ret,year,month,mkt,res,logret,mom,ic,res_vol,z_mom,alpha
0,10001,2011-01-13,EGAS,10.66000,0.004713,2011,1,-0.001435,0.003740,0.004702,0.096737,0.311389,0.005458,-0.057021,-0.000097
1,10001,2011-01-14,EGAS,10.77000,0.010319,2011,1,0.006795,0.007075,0.010266,0.076216,0.321000,0.005463,-1.480040,-0.002596
2,10001,2011-01-18,EGAS,10.66000,-0.010214,2011,1,0.002985,-0.011892,-0.010267,0.084216,0.235195,0.005835,-0.987468,-0.001355
3,10001,2011-01-19,EGAS,10.73000,0.006567,2011,1,-0.012552,0.004792,0.006546,0.087339,0.239741,0.005421,0.074441,0.000097
4,10001,2011-01-20,EGAS,10.76000,0.002796,2011,1,-0.002621,0.000910,0.002792,0.074029,0.221657,0.005383,2.297996,0.002742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11478495,93436,2023-12-22,TSLA,252.53999,-0.007701,2023,12,0.002534,-0.011243,-0.007731,0.683641,0.038838,0.020825,0.609308,0.000493
11478496,93436,2023-12-26,TSLA,256.60999,0.016116,2023,12,0.004689,0.009250,0.015988,0.650205,-0.074504,0.019425,0.330177,-0.000478
11478497,93436,2023-12-27,TSLA,261.44000,0.018822,2023,12,0.002297,0.016981,0.018647,0.691532,-0.026724,0.019668,-0.037313,0.000020
11478498,93436,2023-12-28,TSLA,253.17999,-0.031594,2023,12,-0.000218,-0.028194,-0.032104,0.690453,-0.034562,0.020605,-0.793832,0.000565


In [170]:
alphas = df.pivot(columns=['permno'], values=['alpha'], index=['caldt'])

alphas

alpha                                                    \
permno         10001     10002     10025     10026     10028     10032   
caldt                                                                    
2011-01-13 -0.000097 -0.004728 -0.001074  0.001418 -0.000151  0.003862   
2011-01-14 -0.002596  0.009533  0.001061  0.002486  0.000060 -0.005978   
2011-01-18 -0.001355  0.000684  0.001261 -0.000408 -0.002605 -0.000018   
2011-01-19  0.000097  0.004009  0.003981 -0.000029 -0.001124  0.001775   
2011-01-20  0.002742 -0.003180 -0.001731  0.000528  0.000156 -0.000053   
...              ...       ...       ...       ...       ...       ...   
2023-12-22       NaN       NaN       NaN  0.000923 -0.005193  0.003199   
2023-12-26       NaN       NaN       NaN  0.001211  0.007426 -0.011368   
2023-12-27       NaN       NaN       NaN  0.000519 -0.008106 -0.003307   
2023-12-28       NaN       NaN       NaN -0.000190  0.024212  0.006764   
2023-12-29       NaN       NaN       NaN  0.000039  0.000660  0.004138   

                                                    ...                  \
permno         10044     10051     10066     10100  ... 93422     93423   
caldt                                               ...                   
2011-01-13 -0.000585 -0.000631       NaN  0.000265  ...   NaN       NaN   
2011-01-14 -0.000500 -0.000022       NaN -0.000944  ...   NaN       NaN   
2011-01-18 -0.002483  0.000213       NaN -0.001277  ...   NaN       NaN   
2011-01-19  0.002178  0.000207       NaN  0.004886  ...   NaN       NaN   
2011-01-20  0.000376 -0.000035       NaN -0.000663  ...   NaN       NaN   
...              ...       ...       ...       ...  ...   ...       ...   
2023-12-22  0.002445       NaN -0.001517       NaN  ...   NaN -0.000708   
2023-12-26  0.000226       NaN -0.001357       NaN  ...   NaN  0.000286   
2023-12-27 -0.003385       NaN -0.000312       NaN  ...   NaN -0.000268   
2023-12-28 -0.000009       NaN -0.001328       NaN  ...   NaN -0.000482   
2023-12-29 -0.000726       NaN  0.000232       NaN  ...   NaN -0.000310   

                                                                        
permno         93426 93428 93429 93430 93433     93434 93435     93436  
caldt                                                                   
2011-01-13       NaN   NaN   NaN   NaN   NaN       NaN   NaN       NaN  
2011-01-14       NaN   NaN   NaN   NaN   NaN       NaN   NaN       NaN  
2011-01-18       NaN   NaN   NaN   NaN   NaN       NaN   NaN       NaN  
2011-01-19       NaN   NaN   NaN   NaN   NaN       NaN   NaN       NaN  
2011-01-20       NaN   NaN   NaN   NaN   NaN       NaN   NaN       NaN  
...              ...   ...   ...   ...   ...       ...   ...       ...  
2023-12-22 -0.000148   NaN   NaN   NaN   NaN -0.010709   NaN  0.000493  
2023-12-26 -0.000497   NaN   NaN   NaN   NaN  0.009033   NaN -0.000478  
2023-12-27 -0.000189   NaN   NaN   NaN   NaN  0.008695   NaN  0.000020  
2023-12-28 -0.000134   NaN   NaN   NaN   NaN  0.027957   NaN  0.000565  
2023-12-29 -0.000532   NaN   NaN   NaN   NaN -0.005338   NaN  0.005006  

[3262 rows x 7118 columns]